In [1]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.1 MB/s 
     |████████████████████████████████| 6.6 MB 57.2 MB/s 
     |████████████████████████████████| 120 kB 85.8 MB/s 


In [2]:
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 4.1 MB/s 
     |████████████████████████████████| 212 kB 68.5 MB/s 
     |████████████████████████████████| 115 kB 84.2 MB/s 
     |████████████████████████████████| 127 kB 93.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


# Importando bibliotecas

In [1]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Audio
from scipy.io import wavfile

In [2]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [3]:
from datasets import load_dataset, load_metric

# Puxa dataset

In [8]:
dataset = load_dataset("superb", "ic")

Generating train split:   0%|          | 0/23132 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3118 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3793 [00:00<?, ? examples/s]

Dataset superb downloaded and prepared to /root/.cache/huggingface/datasets/superb/ic/1.9.0/b8183f71eabe8c559d7f3f528ab37a6a21ad1ee088fd3423574cecad8b3ec67e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'speaker_id', 'text', 'action', 'object', 'location'],
        num_rows: 23132
    })
    validation: Dataset({
        features: ['file', 'audio', 'speaker_id', 'text', 'action', 'object', 'location'],
        num_rows: 3118
    })
    test: Dataset({
        features: ['file', 'audio', 'speaker_id', 'text', 'action', 'object', 'location'],
        num_rows: 3793
    })
})

In [10]:

labels = dataset["train"].features["action"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [11]:
id2label

{'0': 'activate',
 '1': 'bring',
 '2': 'change language',
 '3': 'deactivate',
 '4': 'decrease',
 '5': 'increase'}

In [12]:
import pandas as pd
data = {'file': dataset['train']['file'], 'audio': dataset['train']['audio'],
        'action':dataset['train']['action'], 'speaker_id':dataset['train']['speaker_id'],}
mid = pd.DataFrame.from_dict(data)
mid.head()

,file,audio,action,speaker_id
0,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,2,2BqVo8kVB2Skwgyb
1,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,0,2BqVo8kVB2Skwgyb
2,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,0,2BqVo8kVB2Skwgyb
3,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,0,2BqVo8kVB2Skwgyb
4,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,3,2BqVo8kVB2Skwgyb


In [13]:
mid['speaker_id'].nunique()

77

In [14]:
dataset.shape

{'train': (23132, 7), 'validation': (3118, 7), 'test': (3793, 7)}

In [16]:
from sklearn.model_selection import train_test_split
perc = 10
X= mid[['file', 'audio']].copy()
y= mid['action']
X_resto, X_treino_1p, y_resto, y_treino_1p = train_test_split(X, y, stratify=y, test_size=(perc/100), random_state=42)

In [17]:
# data = {'texto': X_treino_1p, 'label': y_treino_1p}
base_treino = pd.DataFrame(X_treino_1p, columns = ['file','audio'])
base_treino['label'] = y_treino_1p
base_treino.shape

(2314, 3)

In [18]:
base_treino.head()

,file,audio,label
9276,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,4
14057,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,1
2795,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,3
13856,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,1
21285,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,5


In [19]:
test =  {'file': dataset['test']['file'], 'audio': dataset['test']['audio'],
        'label':dataset['test']['action'],}
test = pd.DataFrame.from_dict(test)
test.head()

,file,audio,label
0,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,0
1,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,3
2,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,2
3,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,3
4,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,0


In [20]:
import pandas as pd
val = {'file': dataset['validation']['file'], 'audio': dataset['validation']['audio'],
        'label':dataset['validation']['action'],}
val = pd.DataFrame.from_dict(val)
val.head()

,file,audio,label
0,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,0
1,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,3
2,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,2
3,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,3
4,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,0


In [21]:
base_treino.to_pickle('/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ic_original.pkl')

In [22]:
test.to_pickle('/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ic_test.pkl')

In [23]:
val.to_pickle('/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ic_validation.pkl')

In [25]:
import pickle


pickle.dump(label2id, open("/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ic_label2id.pkl", "wb"))  # save it into a file named save.p
pickle.dump(id2label, open("/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ic_id2label.pkl", "wb"))  # save it into a file named save.p

# -------------------------------------------------------------
# Load the dictionary back from the pickle file.
# import pickle

# favorite_color = pickle.load(open("save.p", "rb"))
# favorite_color is now {"lion": "yellow", "kitty": "red"}

# Rodando Classificador na Original

In [4]:
import pickle
label2id = pickle.load(open("/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ic_label2id.pkl", "rb"))
id2label = pickle.load(open("/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ic_id2label.pkl", "rb"))

In [5]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [6]:
import pandas as pd
train = pd.read_pickle('/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ic_original.pkl')

In [7]:
def pega_tamanho_audio(x):
  return x['array'].shape[0]

train['audio'].apply(lambda x:pega_tamanho_audio(x)).max()/16000

6.8266875

In [8]:
train.shape

(2314, 3)

In [9]:
train.label.value_counts(normalize=True)

5    0.257563
4    0.246327
0    0.165082
3    0.133535
2    0.107174
1    0.090320
Name: label, dtype: float64

In [10]:
id2label

{'0': 'activate',
 '1': 'bring',
 '2': 'change language',
 '3': 'deactivate',
 '4': 'decrease',
 '5': 'increase'}

In [11]:
test = pd.read_pickle('/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ic_test.pkl')

In [12]:
val = pd.read_pickle('/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ic_validation.pkl')

In [13]:
from datasets import Dataset
train = Dataset.from_pandas(train)
test = Dataset.from_pandas(test)
val = Dataset.from_pandas(val)

In [14]:
max_duration = 6.0
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, 
        sampling_rate=feature_extractor.sampling_rate, 
        max_length=int(feature_extractor.sampling_rate * max_duration), 
        truncation=True, 
    )
    return inputs

In [15]:
encoded_train = train.map(preprocess_function, remove_columns=["audio", "file"], batched=True)

encoded_test = test.map(preprocess_function, remove_columns=["audio", "file"], batched=True)

encoded_val = val.map(preprocess_function, remove_columns=["audio", "file"], batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:168: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [16]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    "facebook/wav2vec2-base", num_labels=num_labels, label2id=label2id, id2label=id2label
)

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForSequenceClassification: ['quantizer.codevectors', 'project_q.bias', 'project_q.weight', 'project_hid.bias', 'quantizer.weight_proj.weight', 'project_hid.weight', 'quantizer.weight_proj.bias']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertFor

In [17]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="no",
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    learning_rate=3e-5,
    warmup_ratio=0.1,
    logging_steps=1,
    num_train_epochs=3,
    metric_for_best_model="accuracy",
)



In [18]:
def compute_metrics(eval_pred):
    acc = load_metric("accuracy")
    f1 = load_metric("f1", average='macro')
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = acc.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1.compute(predictions=predictions, references=labels, average="macro")["f1"]
    return {"accuracy": accuracy, "f1": f1}

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train,
    eval_dataset=encoded_val,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
)



In [20]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `Wav2Vec2ForSequenceClassification.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `Wav2Vec2ForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2314
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 4
  Total optimization steps = 54


Epoch,Training Loss,Validation Loss,Accuracy,F1
0,1.717100,1.684900,0.263631,0.070063
1,1.584600,1.579155,0.305965,0.165650
2,1.543700,1.528623,0.342207,0.230237


***** Running Evaluation *****
  Num examples = 3118
  Batch size = 32
***** Running Evaluation *****
  Num examples = 3118
  Batch size = 32
***** Running Evaluation *****
  Num examples = 3118
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=54, training_loss=1.6739477471069053, metrics={'train_runtime': 571.3259, 'train_samples_per_second': 12.151, 'train_steps_per_second': 0.095, 'total_flos': 2.4502699763018675e+17, 'train_loss': 1.6739477471069053, 'epoch': 2.99})

In [21]:
predictions = trainer.predict(encoded_test)
print(predictions.predictions.shape, predictions.label_ids.shape)

***** Running Prediction *****
  Num examples = 3793
  Batch size = 32


(3793, 6) (3793,)


In [22]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [23]:
acc = load_metric("accuracy")
f1 = load_metric("f1", average='macro')



In [24]:
accuracy = acc.compute(predictions=preds, references=predictions.label_ids)["accuracy"]
accuracy

0.35486422356973374

In [25]:
f1 = f1.compute(predictions=preds, references=predictions.label_ids, average="macro")["f1"]
f1

0.25232414752855775